In [219]:
# RSA simple implementing
# 
# 质数/素数
#   只能被1和自身整除
#
# 互质
#   最大公约数为1的数
#
# 欧拉函数
#   欧拉函数φ(n)是小于或等于正整数n的数字中与n互质的数的数目
#   如果n是两个质数p和q的乘积, 那么:
#   φ(n) = φ(pq) = φ(p)φ(q) = (p - 1)(q - 1)
#
# 模反元素
#   如果两个正整数a和n互质，那么一定可以找到整数b，使得 ab-1 被n整除，或者说ab被n除的余数是1
#   ab ≡ 1(mod n)  
#
# 欧拉定理
#   如果两个正整数a和n互质，则n的欧拉函数 φ(n) 可以让下面的等式成立：
#   a^φ(n) ≡ 1(mod n)
#   a * a^(φ(n) - 1) ≡ 1(mod n)
#   由此可得：a的φ(n) - 1次方肯定是a关于n的模反元素
#   欧拉定理就可以用来证明模反元素必然存在
#
# 费马小定理
#   假设正整数a与质数p互质，因为质数p的φ(p)等于p-1，则欧拉定理可以写成
#   a^(p-1) ≡ 1 (mod p)
#   费马小定理是欧拉定理的一个特例
#
#
# RSA generator steps
# step1: p, q : two different primes
#   n = p * q  : modulus
# step2:
#   φ(n) = φ(pq) = φ(p)φ(q) = (p - 1) * (q - 1)
#   r = φ(n) = φ(pq) = lcm(p - 1, q - 1)
# step3: 
#   e * d ≡ 1 (mod r) -> e * d % r = 1 
#   e: 1 < e < r and gdc(e, r) == 1
# step4:
#   d: 1 < d < r and d * e mod r == 1
# step5:
#   (e, n) : public key
#   (d, n) : private key
#
# RSA encrypt and decrypt
#   m: message, m_e: encrypted message, m_e_d: decrypted message from m_e
# 
#   m_e ≡ m^e (mod n)
#   m_e_d ≡ m_e^d (mod n) ≡ m^(ed) (mod n)
#   ed ≡ 1 (mod φ(n)) -> ed = hφ(n) + 1
#   m_e_d ≡ m^(hφ(n) + 1) (mod n) ≡ m * m^(hφ(n)) ≡ m * (m^φ(n))^h ≡ m * (1)^h (mod n) ≡ m
# 
# reference:
#   https://zh.wikipedia.org/wiki/RSA%E5%8A%A0%E5%AF%86%E6%BC%94%E7%AE%97%E6%B3%95
#   https://en.wikipedia.org/wiki/RSA_(cryptosystem)
#   https://zhuanlan.zhihu.com/p/33580225
#

In [246]:
%%latex
\begin{equation}
欧拉函数: \phi(n) = \phi(pq) = \phi(p) \times \phi(q) = (p - 1) \times (q -1) \\
r = \phi(n), 1 < e < r, 1 < d < r \\
模反元素: ed \equiv 1 \pmod{r} \equiv 1 \pmod{\phi(n)} \\
\\
欧拉定理: m^{\phi(n)} \equiv 1 \pmod{n} \\
\textit{欧拉定理可以证明模反元素必然存在}: m^{\phi(n)} = m \times m^{\phi(n) - 1} \equiv 1 \pmod{n} \\
\\
m: message \space (m < n)\\
encrypt: m_e = m^{e} \bmod{n} \\
decrypt: m_{ed} = m_e^{d} \bmod{n} \\
prove: m_e^{d} \bmod{n} = m^{ed} \bmod{n} = m^{h\phi(n) + 1} \bmod{n} = m \times m^{h\phi(n)} \bmod{n} = m \times 1 = m
\end{equation}

<IPython.core.display.Latex object>

In [221]:
IGNORE_FIRST_N_PRIMES = 5
P_AND_Q_MIN_DIFF = 5
E_AND_D_MIN_DIFF = 5

In [222]:
def primes_generator():
    primes = []   # primes generated so far
    last = 1      # last number tried
    while True:
        last += 1
        for p in primes:
            if last % p == 0:
                break
        else:
            primes.append(last)
            yield last

# greatest common divisor
def gcd(x, y):
   while(y):
       x, y = y, x % y
        
   return x

# least common multiple
def lcm(x, y):
   return (x * y) // gcd(x, y)

In [223]:
def get_p_q():
    primes = primes_generator()
    for i in range(1, IGNORE_FIRST_N_PRIMES):
        next(primes)   
    p = next(primes)

    for i in range(1, P_AND_Q_MIN_DIFF):
        next(primes)
    q = next(primes)
    
    return (p, q)
        
def get_n(p, q):
    return p * q


def get_r(p, q):
    return lcm(p - 1, q - 1)


def get_e(r):
    for e in range(2, r):
        if gcd(e, r) == 1:
            return e
        else:
            continue
            
    return 0

    
def get_d(r, e):
    for d in range(2, r):
        if d == e and abs(d - e) < E_AND_D_MIN_DIFF:
            continue
            
        if e * d % r == 1:
            return d
        else:
            continue
            
    return 0

In [224]:
def get_rsa(p, q):
    rsa = dict()
    rsa['p'] = p
    rsa['q'] = q
    rsa['n'] = p * q
    
    rsa['r'] = get_r(p, q)
    r = rsa['r']
    e = get_e(r)
    if e == 0:
        rsa['e'] = 0
        rsa['ready'] = False
        return rsa
    
    d = get_d(r, e)
    if d == 0:
        rsa['d'] = 0        
        rsa['ready'] = False
        return rsa
    
    rsa['e'] = e
    rsa['d'] = d
    rsa['ready'] = True
    
    return rsa

In [241]:
def verify_rsa(rsa, msgs):
    for e, d in (('e', 'd'), ('d', 'e')):
        print("test: m -> {}(m) -> {}({}(m)) -> m ...".format(e, d, e))
        for m in msgs:
            m_e = m ** rsa[e] % rsa['n']
            m_e_d = m_e ** rsa[d] % rsa['n']
            print("{status}: {m:4d} -> {m_e:4d} -> {m_e_d:4d}".format(
                status=('PASS' if m == m_e_d else 'FAIL'), 
                m=m, 
                m_e=m_e, 
                m_e_d=m_e_d)
            )
        print("")

In [242]:
def main():
    p, q = get_p_q()
    rsa = get_rsa(p, q)
    print('RSA -> {}'.format(rsa))
    if rsa['ready'] == False:
        print('Get RSA failed!')
        return
    
    CASE_COUNT = 20
    print('RSA verify:')
    print("m < rsa['n'] cases:")
    verify_rsa(rsa, range(max(rsa['n'] - CASE_COUNT, 1), rsa['n']))
    print("m >= rsa['n'] cases:")
    verify_rsa(rsa, range(rsa['n'], rsa['n'] + CASE_COUNT))

In [243]:
main()

RSA -> {'p': 11, 'q': 29, 'n': 319, 'r': 140, 'e': 3, 'd': 47, 'ready': True}
RSA verify:
m < rsa['n'] cases:
test: m -> e(m) -> d(e(m)) -> m ...
PASS:  299 ->  294 ->  299
PASS:  300 ->  159 ->  300
PASS:  301 ->  229 ->  301
PASS:  302 ->  191 ->  302
PASS:  303 ->   51 ->  303
PASS:  304 ->  134 ->  304
PASS:  305 ->  127 ->  305
PASS:  306 ->   36 ->  306
PASS:  307 ->  186 ->  307
PASS:  308 ->  264 ->  308
PASS:  309 ->  276 ->  309
PASS:  310 ->  228 ->  310
PASS:  311 ->  126 ->  311
PASS:  312 ->  295 ->  312
PASS:  313 ->  103 ->  313
PASS:  314 ->  194 ->  314
PASS:  315 ->  255 ->  315
PASS:  316 ->  292 ->  316
PASS:  317 ->  311 ->  317
PASS:  318 ->  318 ->  318

test: m -> d(m) -> e(d(m)) -> m ...
PASS:  299 ->  150 ->  299
PASS:  300 ->  108 ->  300
PASS:  301 ->  247 ->  301
PASS:  302 ->  278 ->  302
PASS:  303 ->  151 ->  303
PASS:  304 ->   39 ->  304
PASS:  305 ->  222 ->  305
PASS:  306 ->   81 ->  306
PASS:  307 ->  186 ->  307
PASS:  308 ->  275 ->  308
PASS:  

In [234]:
print('pass' if True else 'failed')

pass
